图像预处理部分
利用os和opencv导入图片库，并对其进行resize，使符合网络需求。
1.导入imgs.zip中的图片以及driver_imgs_list.csv
#listdir的参数是文件夹的路径
#resize可以缩放图片可参考如下
http://www.xuebuyuan.com/1971769.html

In [1]:
import os
import shutil
import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Lambda, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.pooling import GlobalAveragePooling2D
from keras.applications import inception_v3,xception,inception_resnet_v2
from keras.applications import InceptionV3,Xception,InceptionResNetV2
from keras.callbacks import Callback,ModelCheckpoint,EarlyStopping,LearningRateScheduler
import keras.backend as kb
from keras.optimizers import SGD,Adam
import h5py


Using TensorFlow backend.


In [5]:
def rmrf_mkdir(dirname):
    if os.path.exists(dirname):
        shutil.rmtree(dirname)
    os.mkdir(dirname)

def transfer_image(driver,targetpath):
    path = "./imgs/train"
    images = datacsv.loc[(datacsv['subject']==driver),['img']]
    classnames = datacsv.loc[(datacsv['subject']==driver),['classname']] 

    for i in range(len(images)):
        resource = "".join(os.path.join(path,classnames.values[i][0],images.values[i][0]))
        target = "".join(os.path.join(targetpath,classnames.values[i][0],images.values[i][0]))
        shutil.copy(resource,target)
    return len(images)

# 根据驾驶员分割训练集和验证集
x_drivers = []
y_drivers = []
datacsv = pd.DataFrame(pd.read_csv('driver_imgs_list.csv'))
x_drivers = datacsv['subject'].unique()[:22]
y_drivers = datacsv['subject'].unique()[22:]

# 重新建立目标文件树
rmrf_mkdir("./data")
print('/data/ recreate') 
os.makedirs("./data/test/val")
for c in ["c0","c1","c2","c3","c4","c5","c6","c7","c8","c9"]: 
    os.makedirs("./data/train/{0}".format(c))
    os.makedirs("./data/validation/{0}".format(c))
print('folders created !') 

# 复制文件到目标文件夹下
tra_nb = 0
val_nb = 0
for driver in x_drivers:
    tra_nb += transfer_image(driver, targetpath="./data/train")
print ("train_data transfer is finished")
for driver in y_drivers:
    val_nb += transfer_image(driver,targetpath="./data/validation/")
print ("validation_data transfer is finished")
for file in os.listdir("./imgs/test/"):
    resource = "".join(os.path.join("./imgs/test/",file))
    target = "".join(os.path.join("./data/test/val",file))
    shutil.copy(resource,target)
tex_nb = len(os.listdir("./data/test/val/"))
print ("test_data transfer is finished")
print (tra_nb, val_nb, tex_nb)

test_data transfer is finished
19407 3017 79726


In [9]:
def lr_scheduler(epoch,model):
    a = epoch % 4
    lr = 0.0001
    if a == 0:
        lr = lr / 10
    elif a == 3:
        lr = lr
    else:
        lr = lr / 100
    print ("learning rate is %s"%lr)
    return lr

def Model_fit(size,MODEL,preprocess_fun,optimizer,History,tune=0,Epochs=20,drop=0.3):
    width = size[0]
    height = size[1]
    input_tensor = Input((height, width, 3))
    x = Lambda(preprocess_fun)(input_tensor)
        
    base_model = MODEL(input_tensor=x, include_top=False, pooling="avg", weights="imagenet")
    new_output = Dropout(drop)(base_model.output)
    new_output = Dense(10, activation="softmax")(new_output)
    new_model = Model(inputs=base_model.input, outputs=new_output)
    if tune:
        new_model.load_weights("./a_%s.hdf5"%MODEL.__name__)
    new_model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    new_model.summary()

    gen = ImageDataGenerator() 
    train_generator = gen.flow_from_directory("./data/train", (width, height), shuffle=True, batch_size=16)
    valid_generator = gen.flow_from_directory("./data/validation", (width, height), shuffle=True, batch_size=16)

    history = LossHistory()
    lrreduce = LearningRateScheduler(lr_scheduler)
    earlystopping = EarlyStopping(monitor="val_loss", min_delta=0.0003, patience=5, verbose=1, mode="min")
    if tune:
        checkpoint = ModelCheckpoint("./%s.hdf5"%MODEL.__name__,monitor="val_loss", mode="min", save_best_only=True, period=1)
    else:
        checkpoint = ModelCheckpoint("./a_%s.hdf5"%MODEL.__name__,monitor="val_loss", mode="min", save_best_only=True, period=1)
   
    model = new_model.fit_generator(train_generator, 1+tra_nb//16, Epochs, verbose=1,
                                    validation_data=valid_generator, callbacks=[history,earlystopping,checkpoint,lrreduce])
    
    History["%s"%MODEL.__name__] = model.history
    return new_model

In [ ]:
#IncetionResNetV2，选出合适的h5py
adam = Adam(lr=0.0001)
new_model = Model_fit((299,299),InceptionResNetV2,inception_resnet_v2.preprocess_input,adam,History,Epochs=15,drop=0.4)

In [ ]:
#在adam基础上利用sgd进行精调，降低val_loss，提高val_acc
sgd = SGD(lr=0.0001, decay=1e-6, momentum=0.9, nesterov=True)
new_model = Model_fit((299,299),InceptionResNetV2,inception_resnet_v2.preprocess_input,sgd,History,tune=1,Epochs=15,drop=0.5)

In [ ]:
#根据模型预测结果
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("./data/test", (32, 32),batch_size=16)
y_pred = new_model.predict_generator(test_generator, 1+79726//16, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

# 存入
df = pd.read_csv('sample_submission.csv')
df = pd.DataFrame(y_pred,df.values[:,0],df.columns[1:])
df.to_csv('pred3.csv')
df.head(10)

In [ ]:
#InceptionV3，选出合适的h5py
adam = Adam(lr=0.0003)
new_model = Model_fit((299,299),InceptionV3,inception_v3.preprocess_input,adam,History,tune=0,drop=0.3)

In [ ]:
#在adam基础上利用sgd进行精调，降低val_loss，提高val_acc
sgd = SGD(lr=0.0005, decay=1e-6, momentum=0.9, nesterov=True)
new_model = Model_fit((299,299),InceptionV3,inception_v3.preprocess_input,sgd,History,tune=1,drop=0.4)

In [ ]:
#根据模型预测结果
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("./data/test", (299, 299),batch_size=16)
y_pred = new_model.predict_generator(test_generator, 1+79726//16, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

# 存入
df = pd.read_csv('sample_submission.csv')
df = pd.DataFrame(y_pred,df.values[:,0],df.columns[1:])
df.to_csv('pred.csv3')
df.head(10)

In [ ]:
#Xception，选出合适的h5py
adam = Adam(lr=0.0001)
new_model = Model_fit((299,299),Xception,xception.preprocess_input,adam,History,tune=0,drop=0.5)

In [ ]:
#在adam基础上利用sgd进行精调，降低val_loss，提高val_acc
sgd = SGD(lr=0.00005, decay=1e-6, momentum=0.9, nesterov=True)
new_model = Model_fit((299,299),Xception,xception.preprocess_input,sgd,History,tune=1,drop=0.5)

In [ ]:
# #根据模型预测结果
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("./data/test", (299, 299),batch_size=16)
# y_pred = new_model.predict_generator(test_generator, 1+79726//16, verbose=1)
# y_pred = y_pred.clip(min=0.005, max=0.995)
# print(y_pred[:3])

acc = []
for i, fname in enumerate(test_generator.filenames):
    name = fname[fname.rfind('/')+1:]
    acc.append( [name,y_pred[i][0],y_pred[i][1],y_pred[i][2],y_pred[i][3],y_pred[i][4],
                 y_pred[i][5],y_pred[i][6],y_pred[i][7],y_pred[i][8],y_pred[i][9]] )

print (acc[:10])

df = pd.DataFrame(acc,columns=['img'] + ['c%d'%i for i in range(10)])
df.head(10)
df = df.sort_values(by='img')
df.to_csv('pred4.csv',index=None, float_format='%.3f')
print("csv saved")
# # 存入
# df = pd.read_csv('sample_submission.csv')
# df = pd.DataFrame(y_pred,df.values[:,0],df.columns[1:])
# df.to_csv('pred3.csv')
# df.head(30)

In [ ]:
#根据模型预测结果
gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("./data/test", (32, 32),batch_size=16)
y_pred = new_model.predict_generator(test_generator, 1+79726//16, verbose=1)
y_pred = y_pred.clip(min=0.005, max=0.995)

# 存入
df = pd.read_csv('sample_submission.csv')
df = pd.DataFrame(y_pred,df.values[:,0],df.columns[1:])
df.to_csv('pred.csv3')
df.head(10)

In [ ]:
# val_loss学习曲线 & val_acc学习曲线
for t in ["InceptionResNetV2","InceptionV3","Xception"]:
    plt.plot(History[t].val_losses)
    plt.xlabel('Times')
    plt.ylabel('Val_Loss')
    plt.show()

    plt.plot(History[t].val_acces)
    plt.xlabel('Times')
    plt.ylabel('Val_acc')
    plt.show()

得到三种模型下各自放开所有层的训练结果及相应预测csv，关于csv进行后期数据整理（这里用平均求和的方式，并取最大值为测试结果）

In [ ]:
#平均结果
df1 = pd.read_csv('pred1.csv')
df2 = pd.read_csv('pred2.csv')
df3 = pd.read_csv('pred3.csv')
result = (df1+df2+df3)/3
result.to_csv('pred.csv')
df.head(10)

In [ ]:
# df = pd.read_csv('sample_submission.csv')

# df1 = pd.read_csv('pred01.csv')
# df2 = pd.read_csv('pred1.csv')
# df3 = pd.read_csv('pred11.csv')
# DF1 = pd.DataFrame(df1.values[:,1:],df.values[:,0],df.columns[1:])
# DF2 = pd.DataFrame(df2.values[:,1:],df.values[:,0],df.columns[1:])
# DF3 = pd.DataFrame(df3.values[:,1:],df.values[:,0],df.columns[1:])
# result = (DF1+DF2+DF3)/3
# print (result)
# result.to_csv('pred.csv')

# # 存入
# df = pd.read_csv('sample_submission.csv')
# for i in range(df.shape[0]):
#     for j in range(1,len(df.columns)):
#         df.set_value(i,df.columns[j], result[i][j-1])
# print (df.values[:,1:])
# print (df.values[:,1:].shape)
# print (df.values[:,0])
# print (df.values[:,0].shape)
# print (df.columns[1:])

In [ ]:
# #Resnet50

# width = 224
# height = 224
# input_tensor = Input((height, width, 3))
# x = Lambda(resnet50.preprocess_input)(input_tensor)
        
# base_model = ResNet50(input_tensor=x, input_shape=(width, height, 3), include_top=False, pooling="avg", weights="imagenet")
# new_output = Dropout(0.3)(base_model.output)
# new_output = Dense(10, activation="softmax")(new_output)
# new_model = Model(inputs=base_model.input, outputs=new_output)
    
# adam = Adam(lr=0.0001)
# sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
# new_model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# new_model.summary()

# gen = ImageDataGenerator() 
# train_generator = gen.flow_from_directory("./data/train", (width, height), shuffle=True,batch_size=16)
# validation_generator = gen.flow_from_directory("./data/validation", (width, height), shuffle=True,batch_size=16)

# history = LossHistory()
# earlystopping = EarlyStopping(monitor="val_loss", min_delta=0.0003, patience=5, verbose=1, mode="min")
# checkpoint = ModelCheckpoint("./ResNet50.hdf5",monitor="val_loss", mode="min", save_best_only=True,period=1)
# model = new_model.fit_generator(train_generator,1+tra_nb//16,epochs=15,verbose=1,
#                                 validation_data=validation_generator,callbacks=[history,earlystopping,checkpoint])
# History["ResNet50"] = model.history

# test_generator = gen.flow_from_directory("./data/test", (width, height),batch_size=16)
# y_pred = new_model.predict_generator(test_generator, 1+tex_nb//16)
# y_pred = y_pred.clip(min=0.005, max=0.995)

# # 读入测试集文件夹下所有图片并对图片进行预处理，用于predict
# df = pd.read_csv("sample_submission.csv")
# for i, fname in enumerate(test_generator.filenames):
#     index = int(fname[fname.rfind('_')+1:fname.rfind('.')])
#     for j in range(1,len(df.columns)):
#         df.set_value(index-1,df.columns[j], y_pred[i][j-1])
        
# df.to_csv('pred0.csv', index=None)
# df.head(10)

# # 读入测试集文件夹下所有图片并对图片进行预处理，用于predict
# df = pd.read_csv('sample_submission.csv')
# for i in range(df.shape[0]):
#     for j in range(1,len(df.columns)):
#         df.set_value(i,df.columns[j], y_pred[i][j-1])

# df.to_csv('pred1.csv', index=None)
# df.head(10)